In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [0]:
class Net(nn.Module):
  
  def __init__(self): # 여기선 사용할 layer들을 정의 해 줘야 한다.
    super(Net, self).__init__() # 상속받고
    
    self.conv1 = nn.Conv2d(1, 6, 5) # conv1 객체 생성 : input ch = 1, output = 6, image = 5*5
    self.conv2 = nn.Conv2d(6,16,5) # conv2 객체 생성 : input ch = 6, output = 16, image = 5*5
    # an affine operation: y = Wx + b
    self.fc1 = nn.Linear(16*4*4, 120) # fc 객체 : w = [400, 120]
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    
  def forward(self, x): # 여기서 feed 받아서 처리 시작한다.
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # max_pool2d (input, size)
    x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2)) 
    
    x = x.view(x.size(0), -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    return x


In [11]:
# 정의된 CNN 사용
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [26]:
# torch 는 mini-batch 만 지원한다. 
# ex) nn.Conv2d는 nSamples * nChannels * Height * width

input = torch.randn(1, 1, 28, 28)
out = net(input)
print(out.size())

torch.Size([1, 10])


In [0]:
target = torch.tensor([3], dtype=torch.long)
loss_fn = nn.CrossEntropyLoss()
err = loss_fn(out, target)
#err.backward() # err을 줄이기위한 회귀를 하겠다 선언

In [20]:
print(net.parameters)

<bound method Module.parameters of Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>


###Update Weight

#### net parameter
레이어를 하나씩 불러와줌

#### f.grad.data
그 레이어에 기울기 값을 가져와줌

####f.data.sub_
w = w - lr * gd 이용해서 

In [0]:
learning_rate = 0.01

for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

In [0]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad() #그라디언트 버퍼 0으로 세팅
ouput = net(input)
loss = loss_fn(out, target)
loss.backward()
optimizer.step()